In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
import pandas as pd
from IPython.display import display
import time
import warnings
from selenium.webdriver.common.keys import Keys
import re

- 김민지 [0:229]
- 박유정 [229:458]
- 송나현 [458:687]
- 이소현 [687:916]
- 최수연 [916:1145]

In [ ]:
start = # 본인의 시작점 입력
end = # 본인의 종료지점 입력

In [ ]:
df1 = pd.read_csv('review_candidates.csv')
df1 = df1[start:end]
df1

In [ ]:
prd_review = pd.DataFrame(columns = ['상품url', '상품명', '분위기', '별점', '리뷰'])
prd_review

In [ ]:
warnings.filterwarnings(action='ignore')

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)
driver.implicitly_wait(3)

for idx in df1.index:
    driver.get(df1['상품url'].loc[idx])

    # 최신순 정렬 클릭
    driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[5]/div/div[1]/div/section[2]/div/div[2]/div/div/div[1]/button[2]').click() 
    time.sleep(2)

    review_list = []
    rating_list = []

    for _ in range(20):

        reviews = driver.find_elements(By.CLASS_NAME, 'production-review-item__description') #리뷰 선택
        stars = driver.find_elements(By.CLASS_NAME, 'production-review-item__writer__info__total-star') #평점 선택

        for review, star in zip(reviews, stars) :
            star = star.get_attribute('aria-label')
            star = float(re.sub(r'[^0-9,.]', '', star))
            
            add_row = pd.DataFrame([{'상품url' : df1['상품url'].loc[idx], 
                                     '상품명' : df1['상품명'].loc[idx],
                                     '분위기' : df1['분위기'].loc[idx],
                                     '별점' : star,
                                     '리뷰' : review.text.strip()}])
            prd_review = prd_review.append(add_row)
            
        driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[5]/div/div[1]/div/section[2]/div/ul/li[11]/button').click()
        time.sleep(2)           

    prd_review.to_csv(f'prd_review_{start}_{idx}.csv', index = False)
    print(idx)
        
driver.close()